<a href="https://colab.research.google.com/github/lucasreis95/ifood-data-analyst-case/blob/main/marketing_analytics_dad_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marketing Analytics Project - Optimization of a marketing campaign

In order to optimize the next marketing campaign, this project was divided into 3 major parts:


*   Descriptive analysis of customers who accepted the pilot campaign offer;
*   Clustering customers by characteristic similarities and understanding these groups;
*   Predictive model for targeting the next marketing campaign



## Import Libs and settings

In [ ]:
# import libs
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn import cluster
from sklearn import metrics
from sklearn.cluster import KMeans
import imblearn
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

In [ ]:
# set display settings
pd.set_option('display.max_columns', 100)

## Data import and first look

Here we will import raw data from github and have our first look on the dataset, checking data types, searching for missing values, looking at descriptive statistics and categories distribution.

In [ ]:
# import raw data from github
url = 'https://raw.githubusercontent.com/ifood/ifood-data-advanced-analytics-test/master/ml_project1_data.csv'
df_raw = pd.read_csv(url)
print('Raw dataframe contains', df_raw.shape[0], 'rows and', df_raw.shape[1], 'columns.')

Raw dataframe contains 2240 rows and 29 columns.


In [ ]:
# check for df first impressions
df_raw.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


In [ ]:
# check data types and search for nulls
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

### Take a look on Numeric Variables

In [ ]:
# check raw df descriptive statistics
df_raw.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ID,2240.0,5592.159821,3246.662198,0.0,2828.25,5458.5,8427.75,11191.0
Year_Birth,2240.0,1968.805804,11.984069,1893.0,1959.00,1970.0,1977.00,1996.0
Income,2216.0,52247.251354,25173.076661,1730.0,35303.00,51381.5,68522.00,666666.0
Kidhome,2240.0,0.444196,0.538398,0.0,0.00,0.0,1.00,2.0
Teenhome,2240.0,0.506250,0.544538,0.0,0.00,0.0,1.00,2.0
Recency,2240.0,49.109375,28.962453,0.0,24.00,49.0,74.00,99.0
MntWines,2240.0,303.935714,336.597393,0.0,23.75,173.5,504.25,1493.0
MntFruits,2240.0,26.302232,39.773434,0.0,1.00,8.0,33.00,199.0
MntMeatProducts,2240.0,166.950000,225.715373,0.0,16.00,67.0,232.00,1725.0
MntFishProducts,2240.0,37.525446,54.628979,0.0,3.00,12.0,50.00,259.0


### Take a look on Categorical Variables

In [ ]:
# check categories on object columns (searching for any inconsistency on categories)
# get all categorical columns
cat_columns_list = [col for col in df_raw.columns if df_raw[col].dtype=='O']
print(cat_columns_list)

['Education', 'Marital_Status', 'Dt_Customer']


In [ ]:
# check categories on Education column
df_raw['Education'].value_counts(dropna = False)

Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: Education, dtype: int64

In [ ]:
# check categories on Marital_Status column
df_raw['Marital_Status'].value_counts(dropna = False)

Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64

In [ ]:
# check categories on Dt_Customer column (which is classified as obj but is a date column)
df_raw['Dt_Customer'].value_counts(dropna = False)

2012-08-31    12
2012-09-12    11
2013-02-14    11
2014-05-12    11
2013-08-20    10
              ..
2012-08-05     1
2012-11-18     1
2013-05-25     1
2013-04-14     1
2014-01-09     1
Name: Dt_Customer, Length: 663, dtype: int64

In [ ]:
# check min and max date on Dt_Customer
min_date = df_raw['Dt_Customer'].min()
max_date = df_raw['Dt_Customer'].max()
print('Minimum and maximum dates on Dt_Customer column are', min_date, 'and', max_date, 'respectively.')

Minimum and maximum dates on Dt_Customer column are 2012-07-30 and 2014-06-29 respectively.


## Data Cleaning and Feature Engineering
Customer information are usually messy, specially if they were collected by humans, as we saw on the step before, we have found points that needs to be cleaned, those points are:

*  Some missing values in *Income*;
*  *Dt_Customer* is not on the correct format;
*  Some columns have categories that can be grouped or renamed;
*  Some new features can be created;
*  Some outliers candidates.

So, let's do those changes!




In [ ]:
# as we are going to change raw df, we do need to create a copy of it
df = df_raw.copy()

### Make some changes on columns names
For a better undestanding

In [ ]:
df_raw.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


In [ ]:
# convert all columns names to lower case
df.columns = map(str.lower, df.columns)

In [ ]:
# change some columns names to a easier understanding
cols_to_rename_dict = {
    'mntwines':'spent_on_wine',
    'mntfruits':'spent_on_fruit',
    'mntmeatproducts':'spent_on_meat',
    'mntfishproducts':'spent_on_fish',
    'mntsweetproducts':'spent_on_sweet',
    'mntgoldprods':'spent_on_gold_products',
    'numdealspurchases':'purchases_deals',
    'numwebpurchases':'purchases_web',
    'numcatalogpurchases':'purchases_catalog',
    'numstorepurchases':'purchases_store',
    'numwebvisitsmonth':'visits_web_month',
    'acceptedcmp1':'accepted_cmp_1',
    'acceptedcmp2':'accepted_cmp_2',
    'acceptedcmp3':'accepted_cmp_3',
    'acceptedcmp4':'accepted_cmp_4',
    'acceptedcmp5':'accepted_cmp_5',
    'z_costcontact':'cost_contact',
    'z_revenue':'revenue'
    }
df = df.rename(columns = cols_to_rename_dict)

### Search for duplicated values where we are not expecting for then

In [ ]:
# search for duplicated rows
duplicated_count_df = df[df.duplicated()].shape[0]
print('There are', duplicated_count_df, 'duplicated rows on data frame.')

There are 0 duplicated rows on data frame.


In [ ]:
# search for duplicated ids
duplicated_count_id = df[df.duplicated(subset=['id'])].shape[0]
print('There are', duplicated_count_id, 'duplicated rows on id column.')

There are 0 duplicated rows on id column.


### Change data types

In [ ]:
# convert Dt_Customer to datetime
df['dt_customer'] = pd.to_datetime(df['dt_customer'])

### Feature Engineering

In [ ]:
# create total ammount of money spent by a customer
df['spent_total'] = df['spent_on_wine']\
                  + df['spent_on_fruit']\
                  + df['spent_on_meat']\
                  + df['spent_on_fish']\
                  + df['spent_on_sweet']

In [ ]:
# create spent on regular products feature
# as our products are divided into gold and regular products and we only have a feature for gold products, let's create
# our regular products feature subtracting total spent by spent on gold products
df['spent_on_regular_products'] = df['spent_total'] - df['spent_on_gold_products']

In [ ]:
# create total number of purchases made by a customer
df['purchases_total'] = df['purchases_web']\
                      + df['purchases_catalog']\
                      + df['purchases_store']

In [ ]:
# create purchases without deal feature
# as our products are purchased by web, catalog or store, and we only have a feature for deal products, let's create
# our no deal products feature subtracting total purchases by deal purchases
df['purchases_no_deals'] = df['purchases_total'] - df['purchases_deals']

In [ ]:
# create average ticket of customer
df['avg_ticket'] = df['spent_total']/df['purchases_total']

In [ ]:
# create total number of campaigns accepted
df['accepted_cmp_total'] = df['accepted_cmp_1']\
                         + df['accepted_cmp_2']\
                         + df['accepted_cmp_3']\
                         + df['accepted_cmp_4']\
                         + df['accepted_cmp_5']

In [ ]:
# create days as customer feature

# create max date possible by summing Dt_Customer(customer's enrollment) and Recency (days since the last purchase)
max_date_possible = df['dt_customer'] + pd.to_timedelta(df['recency'], unit='d')
max_date_possible = max_date_possible.max()
# using max date as last day, create days_as_customer feature
df['days_as_customer'] = max_date_possible- df['dt_customer']
df['days_as_customer'] = pd.to_numeric(df['days_as_customer'].dt.days, downcast='integer')

In [ ]:
# create avg monthly spend
df['avg_monthly_spend'] = df['spent_total']/(df['days_as_customer']/30)

In [ ]:
# create pct_of_income_spent_monthly feature
df['avg_pct_of_income_spent_monthly'] = (df['avg_monthly_spend']*100)/(df['income']/12)

In [ ]:
# create avg monthly purchases
df['avg_monthly_purchases'] = df['purchases_total']/(df['days_as_customer']/30)

In [ ]:
# age feature
# get year from max date possible, this will be our 'today' no calculate age feature
max_date_possible_year = max_date_possible.year
df['age'] = max_date_possible_year - df['year_birth']

In [ ]:
# create years of study feature from education
# considering brazilian format of education, we will make some aproximations for years of study based on educational level
df['years_of_study'] = 0
df.loc[df['education'] == 'Basic', 'years_of_study'] = 9
df.loc[df['education'] == '2n Cycle', 'years_of_study'] = 12
df.loc[df['education'] == 'Graduation', 'years_of_study'] = 17
df.loc[df['education'] == 'Master', 'years_of_study'] = 20
df.loc[df['education'] == 'PhD', 'years_of_study'] = 26

In [ ]:
# grouping marital status in together and not together
# create dictionary of categories to replace
marital_status_repl_dict = {
    'Married':1,
    'Together':1,
    'Single':0,
    'Divorced':0,
    'Widow':0,
    'Alone':0,
    'Absurd':0,
    'YOLO':0
                          }
# create a flag column
df['is_together'] = df['marital_status'].replace(marital_status_repl_dict)

In [ ]:
# create total sons and daughters feature
df['total_sons'] = df['kidhome'] + df['teenhome']

In [ ]:
# create is parent flag feature
df['is_parent'] = 0
df.loc[df['total_sons'] != 0, 'is_parent'] = 1

In [ ]:
# create family size feature
# considering people classified as 'together' are two people living together
# and considering the sons are living with the parent who aswered the survey
df['family_size'] = df['is_together']
df['family_size'] = df['family_size'].replace({0:1, 1:2})
df['family_size'] = df['family_size'] + df['total_sons']

### Remove inconsistent rows with created columns
As we have 5 types of products to sell (wines, meat, fruits, fish and sweet) and they are divided into two categories, gold and regular products, there is impossible to spend more in one category than in the sum of total spent.

The same logic is valid to qte of itens purchased with and without deals.

So these values ​​were probably written wrong, and we will delete than.

In [ ]:
# delete inconsistent rows on spent metrics
df_before_spent_inconsistencys = df.copy()
df = df[df['spent_on_regular_products'] >= 0]
print(df_before_spent_inconsistencys.shape[0] - df.shape[0], 'incosistent rows were deleted.')

4 incosistent rows were deleted.


In [ ]:
# delete inconsistent rows on purchase metrics
df_before_purchases_inconsistencys = df.copy()
df = df[df['purchases_no_deals'] >= 0]
print(df_before_purchases_inconsistencys.shape[0] - df.shape[0], 'incosistent rows were deleted.')

3 incosistent rows were deleted.


It's impossible to have purchased 0 products while spend some money, they reverse is true as well.

In [ ]:
# delete inconsistent rows on purchase or spent == 0
df_before_purchases_inconsistencys = df.copy()
df = df[~((df['purchases_total'] == 0) & (df['spent_total'] > 0))]
df = df[~((df['spent_total'] == 0) & (df['purchases_total'] > 0))]
print(df_before_purchases_inconsistencys.shape[0] - df.shape[0], 'incosistent rows were deleted.')

4 incosistent rows were deleted.


### Remove some redundant columns
As we have created lots of features, most of then which are just another way to represent an pre existing feature, some of then are redundant, so let's remove than from data frame.

(And that's a good oportunity to reorder column in a more friendly way)

In [ ]:
# select columns to keep
columns_to_keep = [
                   'years_of_study', 'income', 'age', 'is_together', 'total_sons', 'is_parent', 'family_size', # personal info
                   'spent_on_wine', 'spent_on_fruit', 'spent_on_meat', 'spent_on_fish', 'spent_on_sweet', # spent info
                   'spent_total', 'spent_on_gold_products', 'spent_on_regular_products', # spent info
                   'purchases_web', 'purchases_catalog', 'purchases_store', # purchase info
                   'purchases_deals', 'purchases_no_deals', 'purchases_total', # purchase info
                   'visits_web_month', 'recency', 'days_as_customer', 'complain', 'avg_ticket', 'avg_monthly_spend', 'avg_pct_of_income_spent_monthly', 'avg_monthly_purchases', # business metrics
                   'accepted_cmp_1', 'accepted_cmp_2', 'accepted_cmp_3', 'accepted_cmp_4', 'accepted_cmp_5', 'accepted_cmp_total', # campaign info
                   'response' # pilot campaign info
                   ]

In [ ]:
df = df[columns_to_keep]

### Treating missing values

In [ ]:
# check which columns contains missing values
df.columns[df.isnull().any()].tolist()

['income', 'avg_pct_of_income_spent_monthly']

In [ ]:
# As income is the only raw that have missing values (avg_pct_of_income_spent_monthly was made from income), let's deep dive on it
# check how many rolls do we have
null_rows_count = df[df['income'].isnull()].shape[0]
total_rows_count = df['income'].shape[0]
null_rows_pct = null_rows_count/total_rows_count
null_rows_pct = round(null_rows_pct*100,2)
print('There are', null_rows_count, 'rows containing missing values in Income column, which represents ', null_rows_pct, '% of total data points.')

There are 23 rows containing missing values in Income column, which represents  1.03 % of total data points.


In [ ]:
# compare response rate of rows where income is empty with all df
resp_rate_nulls = df[df['income'].isnull()]['response'].mean()
resp_rate_all = df['response'].mean()
print('Response rate where income is null:', round(resp_rate_nulls * 100, 2), '%')
print('Response rate on all data points:', round(resp_rate_all * 100, 2), '%')

Response rate where income is null: 4.35 %
Response rate on all data points: 14.98 %


We noticed that Response Rate (which will be our response variable on model) is very different on data points where income is null, so it is a good idea fill then.

In [ ]:
# check if there is any feature that is high correlated with Income.
df.corr()[['income']].sort_values(by = 'income', ascending = False)[1:5]

,income
spent_total,0.676405
spent_on_regular_products,0.671678
purchases_no_deals,0.658585
avg_ticket,0.642744


In [ ]:
# as spent_total is high correlated with income, we will use this correlation to fill missing values
# create and auxiliar column that we will create categorical feature to spent_total based on percentiles bins
df['spent_total_binned'] = pd.qcut(df['spent_total'],
                                   q=4,
                                   labels = ['1q', '2q', '3q', '4q'])

In [ ]:
# fill income empty values with the bin median (to avoid outliers) which was created with spent_total feature
df['income'] = df['income'].fillna(df.groupby('spent_total_binned')['income'].transform('median'))
df['avg_pct_of_income_spent_monthly'] = df['avg_pct_of_income_spent_monthly'].fillna(df.groupby('spent_total_binned')['avg_pct_of_income_spent_monthly'].transform('median'))

In [ ]:
# drop auxiliar bin column
df = df.drop(columns = 'spent_total_binned')

In [ ]:
# check if all columns contains missing values was filled, the result should be an empty list
df.columns[df.isnull().any()].tolist()

[]

### Remove Outliers
We used IQR (Inter quartile range) method to remove outliers of our data.

In [ ]:
# check if there is something unusual on data, especially comparing min and max values with mean and 50% percentile
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
years_of_study,2229.0,18.797667,4.432404,9.000000,17.000000,17.000000,20.000000,26.000000
income,2229.0,52288.719157,24812.230638,2447.000000,35441.000000,51390.000000,68487.000000,666666.000000
age,2229.0,45.209062,11.986089,18.000000,37.000000,44.000000,55.000000,121.000000
is_together,2229.0,0.646478,0.478170,0.000000,0.000000,1.000000,1.000000,1.000000
total_sons,2229.0,0.951996,0.751379,0.000000,0.000000,1.000000,1.000000,3.000000
is_parent,2229.0,0.716465,0.450815,0.000000,0.000000,1.000000,1.000000,1.000000
family_size,2229.0,2.598475,0.905210,1.000000,2.000000,3.000000,3.000000,5.000000
spent_on_wine,2229.0,305.321220,336.840098,0.000000,24.000000,177.000000,505.000000,1493.000000
spent_on_fruit,2229.0,26.412741,39.838511,0.000000,2.000000,8.000000,33.000000,199.000000
spent_on_meat,2229.0,167.707044,226.009487,0.000000,16.000000,68.000000,233.000000,1725.000000


In [ ]:
# function that removes outliers from df
def remove_outliers_iqr(df_name, column_name, iqr_multiplier):
  # defining first and third quartile
  q1 = df_name[column_name].quantile(0.25)
  q3 = df_name[column_name].quantile(0.75)

  # creating inter quartile variable
  iqr = q3 - q1

  # creating lower and upper limit
  lower_limit = q1 - iqr_multiplier * iqr
  upper_limit = q3 + iqr_multiplier * iqr

  # remove outliers from df
  df_wo_outliers = df_name[~((df_name[column_name] < lower_limit) | (df_name[column_name] > upper_limit))]

  # inform how many columns were deleted
  rows_before = df_name.shape[0]
  rows_after = df_wo_outliers.shape[0]
  print(rows_before - rows_after, 'rows contains outliers in', column_name, 'column were deleted.')

  # returns df without outliers
  return df_wo_outliers

In [ ]:
# define columns that we want to search for outliers, based on .describe information and business knowledge
cols_to_search_outliers = ['income', 'age','spent_total', 'purchases_total']

In [ ]:
# delete rows containing outliers with iqr multiplier = 1.5
df_before_outliers_removal = df.copy()
for col in cols_to_search_outliers:
  df = remove_outliers_iqr(df_name = df, column_name = col, iqr_multiplier = 1.5)

6 rows contains outliers in income column were deleted.
3 rows contains outliers in age column were deleted.
3 rows contains outliers in spent_total column were deleted.
0 rows contains outliers in purchases_total column were deleted.


In [ ]:
# remove outliers from spent_on_meat by using higher iqr multiplier
# during the EDA we saw that remained some really strong outliers, so we decided to come back and remove then
df = remove_outliers_iqr(df_name = df, column_name = 'spent_on_meat', iqr_multiplier = 4)
df = remove_outliers_iqr(df_name = df, column_name = 'spent_on_sweet', iqr_multiplier = 6)
df = remove_outliers_iqr(df_name = df, column_name = 'avg_ticket', iqr_multiplier = 6)

2 rows contains outliers in spent_on_meat column were deleted.
1 rows contains outliers in spent_on_sweet column were deleted.
0 rows contains outliers in avg_ticket column were deleted.


In [ ]:
print('We removed', df_before_outliers_removal.shape[0] - df.shape[0], 'data points containing outliers!')

We removed 15 data points containing outliers!


## Exploratory Data Analysis

Now that we have done a basic data cleaning and feature engineering, we are able to do some in depth EDA before build a model to predict customers who are more likely to purchase the offer.

In [ ]:
# Setting color patterns
primary_color = 'rgba(188, 115, 171, 100)' # purple
secundary_color = 'rgba(251, 221, 228, 100)' # pink
third_color = 'rgba(240, 240, 240, 100)' # gray

### How was the first 5 campaigns and the pilot campaign?

As we don't have any information about when those previus campaigns were launched, we will assume all customers who were contacted was already clients since the launch of the first campaign.

In [ ]:
# function that calculate sucess rate of a campaign
def define_campaign_sucess_rate(df_name, campaign_column):
  return round(df_name[campaign_column].value_counts(normalize = True).iloc[1], 3)

In [ ]:
# define sucess rate (sr) for all previus campaigns
sr_cmp_1 = define_campaign_sucess_rate(df, 'accepted_cmp_1')
sr_cmp_2 = define_campaign_sucess_rate(df, 'accepted_cmp_2')
sr_cmp_3 = define_campaign_sucess_rate(df, 'accepted_cmp_3')
sr_cmp_4 = define_campaign_sucess_rate(df, 'accepted_cmp_4')
sr_cmp_5 = define_campaign_sucess_rate(df, 'accepted_cmp_5')

# create lists with sucess rates and campaings labels
sucess_rate_list = [sr_cmp_1, sr_cmp_2, sr_cmp_3, sr_cmp_4, sr_cmp_5]
cmp_label_list = ['cmp 1', 'cmp 2', 'cmp 3', 'cmp 4', 'cmp 5']
data_sr_cmp = {'Campaing' : cmp_label_list, 'Sucess Rate' : sucess_rate_list}

# create sucess rates campaign data frame that will be ploted
df_sr_cmp = pd.DataFrame(data_sr_cmp)

In [ ]:
# plot line chart with sucess rates among the different campaings
# line plot setting
fig = px.line(
              data_frame = df_sr_cmp,
              x='Campaing',
              y='Sucess Rate',
              title='<b>Sucess Rate of previus campaigns',
              text = 'Sucess Rate',
              width=1000,
              height=600
              )

# traces settings
fig.update_traces(
                  texttemplate='%{y}',
                  textposition='top center',
                  line_color = primary_color
                 )

# layout settings
fig.update_layout(
                  yaxis_tickformat='.1%',
                  font_family='sans-serif',
                  title_font_size = 20,
                  font_size = 16,
                  plot_bgcolor = third_color,
                  )

# plot
fig.show()

### Histograms and Chi-Square Test
Here we will test variables dependencies with response, to check which variables impacts response rate and inside each of those variables, which categories impacts response rate too.

In [ ]:
# function that tests dependence between two variables
def chi2_test(df_name_c2, var1_c2, var2_c2, probability_c2, bins_c2):
  # bin column (as we are dealing with numeric columns)
  binned_column = pd.cut(df_name_c2[var2_c2], bins = bins_c2, include_lowest = True)

  # create contingency table
  contigency = pd.crosstab(index = df_name_c2[var1_c2],columns =  binned_column).to_numpy()

  # calculate chi2 test
  stat, p, dof, expected = chi2_contingency(contigency)

  # interpret chi2 test and return the result
  critical = chi2.ppf(probability_c2, dof)
  if abs(stat) >= critical:
    return 'Chi2 test result: Variables are dependent'
  else:
    return 'Chi2 test result: Variables are independent'

In [ ]:
# function that creates histogram with full stacked bars
def create_response_histogram(df_name_hist, var1_hist, var2_hist, n_bins_hist, chi2_test_result):
  # defining plot settings
  fig = px.histogram(
    df_name_hist,
    x = var2_hist,
    color = var1_hist,
    nbins = n_bins_hist,
    barnorm = 'fraction',
    color_discrete_map = {0:secundary_color , 1:primary_color},
    text_auto = '.0%',
    width = 800,
    height = 450,
    title = '<b>Response Rate to pilot campaign<br>on each category/bin of ' + var2_hist.replace('_',' ')
                  )

  # defining layout settings
  fig = fig.update_layout(
      bargap = 0.1,
      font_size = 16,
      title_font_size = 20,
      font_family = 'sans-serif',
      xaxis_title = var2_hist.replace('_',' '),
      yaxis_title = 'percentage',
      yaxis_tickformat = ',.0%',
      plot_bgcolor = third_color,
      uniformtext_minsize = 12,
      uniformtext_mode = 'hide',
      annotations = \
                      [{'xref':'paper', 'yref':'paper', 'x':0.5, 'y':-0.3,
                        'xanchor':'center', 'yanchor':'bottom',
                        'text':chi2_test_result,
                        'font':dict(size=12, color='gray'),
                        'showarrow':False}]
                        )

  # create horizontal line that will represent mean (fow now, it's a static value of 0.15)
  fig = fig.add_hline(y = 0.15, opacity = 0.2, line_dash = 'dash')
  return fig

In [ ]:
# joining chi2_test function with create_response_histogram function
# to plot the chi2 test result inside de histogram plot

# This function works properly in two steps, the first is determining the number of bins through the parameter 'n_bins'
# and the second is determining the range of each bin after executing the function with random values ​​in the parameter 'bins' and making a visual analysis of the size of each bin

# function that plot histogram with chi2 test result
def get_plot_and_chi2(df_name, var1, var2, n_bins, probability, bins):
  # chi2_test function
  c2_result = chi2_test(
      df_name_c2 = df_name,
      var1_c2 = var1,
      var2_c2 = var2,
      probability_c2 = probability,
      bins_c2 = bins)

  # create_response_histogram function
  plt = create_response_histogram(
      df_name_hist = df_name,
      var1_hist = var1,
      var2_hist = var2,
      n_bins_hist = n_bins,
      chi2_test_result = c2_result)
  return plt

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'years_of_study', n_bins = 5, probability = 0.95, bins = [-0.1,10,15,20,25,30])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'income', n_bins = 6, probability = 0.95, bins = [-0.1,20000,40000,60000,80000,100000,120000])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'age', n_bins = 4, probability = 0.95, bins = [-0.1, 19, 39, 59, 79])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'is_together', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1.1])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'is_parent', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1.1])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'total_sons', n_bins = 4, probability = 0.95, bins = [-0.1, 1, 2, 3, 4])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'family_size', n_bins = 5, probability = 0.95, bins = [-0.1, 1, 2, 3, 4, 5])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_total', n_bins = 5, probability = 0.95, bins = [-0.1, 499, 999, 1499, 1999, 2499])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_on_gold_products', n_bins = 5, probability = 0.95, bins = [-0.1, 49, 99, 149, 199, 249])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_on_regular_products', n_bins = 5, probability = 0.95, bins = [-0.1, 499, 999, 1499, 1999, 2499])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_on_wine', n_bins = 8, probability = 0.95, bins = [-0.1, 399, 599, 799, 999, 1199, 1399, 1599])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_on_fruit', n_bins = 4, probability = 0.95, bins = [-0.1, 49, 99, 149, 199])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_on_meat', n_bins = 5, probability = 0.95, bins = [-0.1, 199, 399, 599, 799, 999])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_on_fish', n_bins = 6, probability = 0.95, bins = [-0.1, 49, 99, 149, 199, 249, 299])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'spent_on_sweet', n_bins = 4, probability = 0.95, bins = [-0.1, 49, 99, 149, 199])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'avg_ticket', n_bins = 4, probability = 0.95, bins = [-0.1, 49.9, 99.9, 149.9, 199.9])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'avg_monthly_spend', n_bins = 4, probability = 0.95, bins = [-0.1, 100, 300, 500, 700])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'avg_pct_of_income_spent_monthly', n_bins = 5, probability = 0.95, bins = [-0.1, 1, 3, 5, 7, 9])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'purchases_total', n_bins = 4, probability = 0.95, bins = [-0.1, 9, 19, 29, 39])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'purchases_web', n_bins = 6, probability = 0.95, bins = [-0.1, 1, 3, 5, 7, 9, 11])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'purchases_catalog', n_bins = 6, probability = 0.95, bins = [-0.1, 1, 3, 5, 7, 9, 11])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'purchases_store', n_bins = 7, probability = 0.95, bins = [-0.1, 1, 3, 5, 7, 9, 11, 13])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'purchases_deals', n_bins = 4, probability = 0.95, bins = [-0.1, 4, 9, 14, 19])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'purchases_no_deals', n_bins = 4, probability = 0.95, bins = [-0.1, 9, 19, 29, 39])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'avg_monthly_purchases', n_bins = 4, probability = 0.95, bins = [-0.1, 1.99, 3.99, 5.99, 7.99])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'accepted_cmp_1', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'accepted_cmp_2', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'accepted_cmp_3', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'accepted_cmp_4', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'accepted_cmp_5', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'accepted_cmp_total', n_bins = 5, probability = 0.95, bins = [-0.1, 0, 1, 2, 3, 4])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'visits_web_month', n_bins = 3, probability = 0.95, bins = [-0.1, 9, 19, 29])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'recency', n_bins = 5, probability = 0.95, bins = [-0.1, 19, 39, 59, 79, 99])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'days_as_customer', n_bins = 4, probability = 0.95, bins = [-0.1, 199, 399, 599, 799])

In [ ]:
get_plot_and_chi2(df_name = df, var1 = 'response', var2 = 'complain', n_bins = 2, probability = 0.95, bins = [-0.1, 0.5, 1])

## Create customer clusters

Here we will understand which groups of similar customers we have in our base.

### Rescaling
Now it's time to **Rescaling** our features, as our plan is to create clusters of out data points, it's a good practice to choose standardisation (mean-0, sigma-1), because with that kind of rescaling you will keep mean and variance equal for all features, so none of then will affect clustering process and you will have a more 'fair' cluster.

In [ ]:
# Rescaling the attributes
df_scaled = df.copy()

# fit_transform
df_scaled = StandardScaler().fit_transform(df_scaled)

# get back values to df
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = df.columns

### Dimensionality reduction
As we will need too many variables to explain (90%) our dataset (17), we decided to keep with original columns and not use PCA for dimensional reduction.

In [ ]:
# function that return how many features do we need to explain a percent of df
def check_feat_qte_pca(df, percent_of_explanation):
  pca = PCA(n_components = percent_of_explanation)
  pca.fit(df_scaled)
  reduced = pca.transform(df_scaled)
  return print('From our', df.shape[1], 'columns dataset, we do need to reduce to', reduced.shape[1], \
               'features dataset to explain', percent_of_explanation*100, '% of dataset variance.')

In [ ]:
check_feat_qte_pca(df_scaled, 0.9)

From our 36 columns dataset, we do need to reduce to 17 features dataset to explain 90.0 % of dataset variance.


### Select number of clusters
With the elbow rule and the silhoutte score we were able to define the best number of clusters.

In [ ]:
# kmeans works better without binary data, so let's remove then
# that's not a big problem why most of then generate another feature
cols_to_remove = ['is_together',
                  'is_parent',
                  'complain',
                  'accepted_cmp_1',
                  'accepted_cmp_2',
                  'accepted_cmp_3',
                  'accepted_cmp_4',
                  'accepted_cmp_5',
                  'response'
                  ]

In [ ]:
# remove binary cols to perform kmeans
df_kmeans = df_scaled.drop(columns = cols_to_remove).copy()

In [ ]:
# create elbow method plot

# create empty distortion list
distortions = []
K = range(1,11)

# calculate distortions for every K (number of clusters)
for cluster_size in K:
    kmeans = cluster.KMeans(n_clusters=cluster_size, init='k-means++')
    kmeans = kmeans.fit(df_kmeans)
    # append distorition to list
    distortions.append(kmeans.inertia_)

# create df to plot
df_plot = pd.DataFrame({'Clusters': K, 'Distortions': distortions})

# plot line chart
fig = px.line(df_plot, x='Clusters', y='Distortions')
fig = fig.update_traces(mode='lines+markers', line_color = primary_color)
fig = fig.update_layout(plot_bgcolor = third_color)
fig.show()

In [ ]:
# create silhoutte score method

# create empty silhoutte score list
silhouette_scores = []
K = range(2,11)

# calculate silhoutte score for every number of cluster
for cluster_size in K:
    kmeans = cluster.KMeans(n_clusters=cluster_size, init='k-means++', random_state=10)
    labels = kmeans.fit(df_kmeans).labels_
    silhouette_score = metrics.silhouette_score(df_kmeans,
                                                labels,
                                                metric='euclidean',
                                                sample_size=1000,
                                                random_state=10)
    # append score to list
    silhouette_scores.append(silhouette_score)


# create df to plot
df_plot = pd.DataFrame({'Clusters': K, 'Silhouette Score': silhouette_scores})

# plot line chart
fig = px.line(df_plot, x='Clusters', y='Silhouette Score')
fig = fig.update_traces(mode='lines+markers', line_color = primary_color)
fig = fig.update_layout(plot_bgcolor = third_color)
fig.show()

### Apply clustering and checking how clusters looks like



In [ ]:
# apply clustering to our df
kmeans = KMeans(n_clusters = 2, random_state = 10)
# create cluster column on our df
df['cluster']  = kmeans.fit_predict(df_kmeans)

In [ ]:
# plot clusters size
fig = px.histogram(df, x = 'cluster', text_auto=True, color_discrete_sequence = [primary_color])
fig = fig.update_layout(
      bargap=0.1,
      height = 500,
      width=500,
       )
fig

In [ ]:
# test how clusters stands for some variables
px.scatter(df, x='income', y='spent_total', color='cluster', height = 400, color_continuous_scale   = ['rgba(134, 8, 252, 100)','rgba(252, 8, 240, 100)'])

In [ ]:
# test how clusters stands for some variables
px.scatter(df, x='age', y='purchases_total', color='cluster', height = 400, color_continuous_scale   = ['rgba(134, 8, 252, 100)','rgba(252, 8, 240, 100)'])

In [ ]:
# test how clusters stands for some variables
px.scatter(df, x='avg_monthly_purchases', y='avg_pct_of_income_spent_monthly', color='cluster', height = 400, color_continuous_scale   = ['rgba(134, 8, 252, 100)','rgba(252, 8, 240, 100)'])

### Compare clusters between themselves


In [ ]:
# create absolute varible comparation between clusters
df_grouped_clusters_abs = df.groupby('cluster').mean().transpose().reset_index()
# renem columns to match business definition
df_grouped_clusters_abs = df_grouped_clusters_abs.rename(index = {'cluster':'idx'},
                                                       columns = {1:'premium',
                                                                  0:'standard',
                                                                 'index':'variable'})
# melt
df_grouped_clusters_abs = pd.melt(df_grouped_clusters_abs, id_vars = ['variable'], value_vars = ['premium', 'standard'])
df_grouped_clusters_abs = df_grouped_clusters_abs.rename(columns  = {'value':'value_absolute'})
df_grouped_clusters_abs = df_grouped_clusters_abs.sort_values(by = ['variable', 'cluster'])

In [ ]:
# create relative varible comparation between clusters
df_grouped_clusters_rlt = df.groupby('cluster').mean().transpose().reset_index()
# renem columns to match business definition
df_grouped_clusters_rlt = df_grouped_clusters_rlt.rename(index = {'cluster':'idx'},
                                                columns = {1:'premium_abs',
                                                           0:'standard_abs',
                                                          'index':'variable'})
# create relative values to clusters means
df_grouped_clusters_rlt['premium'] = df_grouped_clusters_rlt['premium_abs']/(df_grouped_clusters_rlt['premium_abs']+df_grouped_clusters_rlt['standard_abs'])
df_grouped_clusters_rlt['standard'] = df_grouped_clusters_rlt['standard_abs']/(df_grouped_clusters_rlt['premium_abs']+df_grouped_clusters_rlt['standard_abs'])
# melt
df_grouped_clusters_rlt = pd.melt(df_grouped_clusters_rlt, id_vars = ['variable'], value_vars = ['premium', 'standard'])
df_grouped_clusters_rlt = df_grouped_clusters_rlt.rename(columns  = {'value':'value_relative'})
df_grouped_clusters_rlt = df_grouped_clusters_rlt.sort_values(by = ['variable', 'cluster'])

In [ ]:
# join absolute and relative values
df_grouped_clusters = df_grouped_clusters_abs.merge(df_grouped_clusters_rlt, on = ['variable', 'cluster'], how = 'inner')
# round values
df_grouped_clusters['value_absolute'] = df_grouped_clusters['value_absolute'].round(1)
df_grouped_clusters['value_relative'] = (df_grouped_clusters['value_relative']*100).round(1)
# create text column
df_grouped_clusters['text'] = df_grouped_clusters['value_absolute'].astype(str) + ' (' + df_grouped_clusters['value_relative'].astype(str) + '%)'

In [ ]:
# select variables that we want to plot
cols_to_plot = [
                'age',
                'family_size',
                'income',
                'spent_total',
                'avg_pct_of_income_spent_monthly',
                'purchases_total',
                'purchases_deals',
                'avg_ticket',
                'visits_web_month',
                'recency',
                'days_as_customer'
              ]

# create df with variables that we want to plot
df_grouped_clusters_plt  = df_grouped_clusters[df_grouped_clusters['variable'].isin(cols_to_plot)]

In [ ]:
# plot bars with means of each variable and a relative comparation between then
fig = px.bar(df_grouped_clusters_plt,
       y='variable',
       x='value_relative',
       color = 'cluster',
       orientation = 'h',
       color_discrete_map = {'premium':'rgba(134, 8, 252, 100)' , 'standard':'rgba(252, 8, 240, 100)'},
       category_orders = {'variable':cols_to_plot},
       text = 'text',
       #text_auto = True,
       height = 700,
       width = 1400
       )

# plot layout
fig = fig.update_layout(
           xaxis_title = 'relative feature comparison between clusters',
           font_family='sans-serif',
           plot_bgcolor = 'white',
           bargap=0.4,
           font_size = 16
                      )

# plot traces
fig = fig.update_traces(textposition='inside',
                        insidetextanchor='middle')

# create horizontal line that will represent 50%
fig = fig.add_vline(x=50, opacity=1, line_dash='dash')
fig

In [ ]:
# rename cluster column to 'is premium', as it's already on a binary format and 1 is already premium cluster
df = df.rename(columns = {'cluster':'is_premium'})

## Create campaign predictive model

Here we will build our predictive model that will indicate to which customers we should send the campaign.

### Feature selection
We will use pearson correlation to select our features, as we saw on EDA, most of the features that are dependent with response, appeared to have some linearity between then.

In [ ]:
# set dfs premium to be modeled
df_premium = df[df['is_premium'] == 1].copy()
df_premium = df_premium.drop(columns = 'is_premium')

# set dfs standard to be modeled
df_standard = df[df['is_premium'] == 0].copy()
df_standard = df_standard.drop(columns = 'is_premium')

In [ ]:
# function that drop correlated features and keep the one wich is more correlated with response variable
def drop_correlated_features(df, response_var):
  # sort columns in order by the most correlated variables with response to less
  ix = df.corr().abs().sort_values(response_var, ascending=False).index
  df_sorted = df.loc[:, ix]

  # Create correlation matrix
  correl = df_sorted.corr().abs()

  # Select upper triangle of correlation matrix
  upper = correl.where(np.triu(np.ones(correl.shape), k=1).astype(bool))

  # Find index of feature columns with correlation greater than 0.5
  to_drop = [column for column in upper.columns if any(upper[column] > 0.5)]

  # Drop features
  df_cleaned = df_sorted.drop(to_drop, axis=1)

  # shape of final df
  shape = df_cleaned.shape
  # print shape of cleaned df and columns that we dropped
  print('After dropping highly correlated features, our has {} records and {} features'.format(shape[0], shape[1]))
  print('Dropped features: ', to_drop)
  return df_cleaned

In [ ]:
# drop correlated features premium cluster
df_premium = drop_correlated_features(df_premium, 'response')

After dropping highly correlated features, our has 905 records and 18 features
Dropped features:  ['accepted_cmp_5', 'accepted_cmp_1', 'spent_on_regular_products', 'avg_ticket', 'is_parent', 'spent_on_wine', 'spent_on_meat', 'total_sons', 'income', 'is_together', 'accepted_cmp_4', 'avg_monthly_purchases', 'purchases_deals', 'purchases_no_deals', 'visits_web_month', 'avg_monthly_spend', 'avg_pct_of_income_spent_monthly', 'purchases_total']


In [ ]:
# drop correlated features standard cluster
df_standard = drop_correlated_features(df_standard, 'response')

After dropping highly correlated features, our has 1309 records and 15 features
Dropped features:  ['accepted_cmp_3', 'purchases_catalog', 'purchases_web', 'accepted_cmp_4', 'spent_on_meat', 'purchases_total', 'avg_monthly_purchases', 'family_size', 'spent_total', 'avg_ticket', 'income', 'spent_on_wine', 'purchases_store', 'spent_on_regular_products', 'spent_on_fruit', 'is_parent', 'avg_monthly_spend', 'spent_on_fish', 'purchases_no_deals', 'total_sons', 'avg_pct_of_income_spent_monthly']


In [ ]:
# plot corr matrix
def plot_corr_matrix(df):
  # check correlation matrix
  fig = px.imshow(
            df_premium.corr().round(2),
            text_auto=True,
            color_continuous_scale = [third_color, primary_color],
            width=700,
            height=700
            )
  return fig

In [ ]:
# plot matrix with the features left on premium df
plot_corr_matrix(df_premium)

In [ ]:
# plot matrix with the features left on standard df
plot_corr_matrix(df_standard)

### Dealing with missing data
As result of dataset cleaning we are left with no questions with NaNs.

In [ ]:
# Finding NANs
def find_nans(df):
  df_null_check = df.isnull().sum().to_frame()
  print('We found', df_null_check[df_null_check[0] > 0].shape[0], 'NaNs on the dataset')

In [ ]:
# find nans on premium df
find_nans(df_premium)

We found 0 NaNs on the dataset


In [ ]:
# find nans on standard df
find_nans(df_standard)

We found 0 NaNs on the dataset


### Split data

In [ ]:
# split test and train df
def split_data(df, test_size, random_state):
  train, test = train_test_split(df, test_size=test_size, random_state=random_state)
  print('Train Shape:', train.shape)
  print('Test Shape:', test.shape)
  return train, test

In [ ]:
# split test and train df for premium cluster
train_prem, test_prem = split_data(df_premium, test_size = 0.2, random_state = 10)

Train Shape: (724, 18)
Test Shape: (181, 18)


In [ ]:
# split test and train df for standard cluster
train_stan, test_stan = split_data(df_standard, test_size = 0.2, random_state = 10)

Train Shape: (1047, 15)
Test Shape: (262, 15)


### Oversampling train set

The classes are imbalanced and it's impacting the logistic regression model. So, let's balance it!

In [ ]:
# function that plot bars showing how imbalanced our df is
def plot_bar_count_var(df, var):
  # create frame to be plotted
  df_to_plot = df['response'].value_counts().to_frame().reset_index().rename(columns = {'index':'response', 'response':'count'})

  # plot df as bars of diferent colors
  fig = px.bar(df_to_plot,
              x = 'response',
              y = 'count',
              text = 'count',
              width=300,
              height=300,
              color = 'response',
              color_continuous_scale = [
                                        secundary_color,
                                        primary_color
                                        ]
              )
  return fig

In [ ]:
# plot class distribution of premium df
plot_bar_count_var(df = train_prem, var = 'response')

In [ ]:
# plot class distribution of standard df
plot_bar_count_var(df = train_stan, var = 'response')

In [ ]:
# function that apply random oversampling method to our train data
def train_df_to_oversample(df_train, response_var):
  # define dataset x and y
  X = df_train.drop(response_var, axis = 1)
  y = df_train[response_var]

  # define oversampling strategy
  oversample = RandomOverSampler(sampling_strategy='minority')

  # fit and apply the transformation
  X_over, y_over = oversample.fit_resample(X, y)
  df_train = X_over.copy()
  df_train[response_var] = y_over.copy()

  return df_train

In [ ]:
# apply oversampling to our dfs
train_prem = train_df_to_oversample(train_prem, 'response').copy()
train_stan = train_df_to_oversample(train_stan, 'response').copy()

In [ ]:
# plot class distribution of premium df after oversampled
plot_bar_count_var(df = train_prem, var = 'response')

In [ ]:
# plot class distribution of standard df after oversampled
plot_bar_count_var(df = train_stan, var = 'response')

### Model training

In [ ]:
# setting X,y train and test, removing target variable
def setting_x_y_train_test(df_train, df_test, var_response):
  # set train X and Y
  ytrain = df_train[var_response].copy()
  Xtrain = df_train.drop([var_response], axis=1).copy()
  # set test X and Y
  ytest = df_test[var_response].copy()
  Xtest = df_test.drop([var_response], axis=1).copy()
  return ytrain, Xtrain, ytest, Xtest

In [ ]:
# set X,y train and test, removing target variable for premium df
ytrain_prem, Xtrain_prem, ytest_prem, Xtest_prem = setting_x_y_train_test(train_prem, test_prem, 'response')
# set X,y train and test, removing target variable for standard df
ytrain_stan, Xtrain_stan, ytest_stan, Xtest_stan = setting_x_y_train_test(train_stan, test_stan, 'response')

In [ ]:
# function that calculates model metrics and display it
def display_scores(predictor, X, y):
    # Getting the predicted values
    ypred = predictor.predict(X)
    ypred_score = predictor.predict_proba(X)

    # calculating metrics
    accuracy = accuracy_score(y, ypred)
    roc = roc_auc_score(y, pd.DataFrame(ypred_score)[1])
    confusion = confusion_matrix(y, ypred)

    # print accuracy score and roc score
    print('Accuracy: ', accuracy)
    print('AUC: ', roc)

    # setting errors type incidents
    type1_error = confusion[0][1] / confusion[0].sum() # False Positive - model predicted customer accepted campaign, while it wasn't!
    type2_error = confusion[1][0] / confusion[1].sum() # False Negative - model predicted response customer dont't accepted campaign, while it was!

    # print errors types incidents and plot confusion matrix
    print('Type 1 error: ', type1_error, '| False Positive: predicted customer accepted the campaign, while it was not!')
    print('Type 2 error: ', type2_error, '| False Negative - predicted response customer do not accepted campaign, while it was!')
    fig = px.imshow(confusion, text_auto=True,color_continuous_scale  = 'PuRd', aspect='auto', height = 600, width = 700)
    fig = fig.update_layout(
      font_size = 20)
    return fig

In [ ]:
# premium
# set model parameters
lreg_prem = LogisticRegression(solver='liblinear', random_state=10)
# Fit the model
lreg_prem.fit(Xtrain_prem, ytrain_prem)
# Check model metrics
display_scores(lreg_prem, Xtrain_prem, ytrain_prem)

Accuracy:  0.8303886925795053
AUC:  0.915017979997253
Type 1 error:  0.15901060070671377 | False Positive: predicted customer accepted the campaign, while it was not!
Type 2 error:  0.18021201413427562 | False Negative - predicted response customer do not accepted campaign, while it was!


In [ ]:
# standard
# set model parameters
lreg_stan = LogisticRegression(solver='liblinear', random_state=10)
# Fit the model
lreg_stan.fit(Xtrain_stan, ytrain_stan)
# Check model metrics
display_scores(lreg_stan, Xtrain_stan, ytrain_stan)

Accuracy:  0.8055260361317748
AUC:  0.8871031676569007
Type 1 error:  0.20085015940488843 | False Positive: predicted customer accepted the campaign, while it was not!
Type 2 error:  0.18809776833156217 | False Negative - predicted response customer do not accepted campaign, while it was!


In [ ]:
# executes cross validation and display scores
def do_cv(predictor, X, y, cv):
    # calculate scores
    cv_score = cross_val_score(predictor, X, y, scoring='roc_auc', cv=5)

    # print metrics
    print ('Mean AUC score after a', cv, 'fold cross validation: ', cv_score.mean())
    print ('AUC score of each fold: ', cv_score)

In [ ]:
# premium
# do Cross Validation
do_cv(lreg_prem, Xtrain_prem, ytrain_prem, 5)
# display scores and plot confusion matrix
display_scores(lreg_prem, Xtrain_prem, ytrain_prem)

Mean AUC score after a 5 fold cross validation:  0.9081406723795155
AUC score of each fold:  [0.89004037 0.91755939 0.91236589 0.90774532 0.9129924 ]
Accuracy:  0.8303886925795053
AUC:  0.915017979997253
Type 1 error:  0.15901060070671377 | False Positive: predicted customer accepted the campaign, while it was not!
Type 2 error:  0.18021201413427562 | False Negative - predicted response customer do not accepted campaign, while it was!


In [ ]:
# stan
# do Cross Validation
do_cv(lreg_stan, Xtrain_stan, ytrain_stan, 5)
# display scores and plot confusion matrix
display_scores(lreg_stan, Xtrain_stan, ytrain_stan)

Mean AUC score after a 5 fold cross validation:  0.8838063262123924
AUC score of each fold:  [0.87162276 0.86904762 0.89049061 0.90678757 0.88108307]
Accuracy:  0.8055260361317748
AUC:  0.8871031676569007
Type 1 error:  0.20085015940488843 | False Positive: predicted customer accepted the campaign, while it was not!
Type 2 error:  0.18809776833156217 | False Negative - predicted response customer do not accepted campaign, while it was!


### Model validation on test data

In [ ]:
# display model scores for premium cluster
display_scores(lreg_prem, Xtest_prem, ytest_prem)

Accuracy:  0.8176795580110497
AUC:  0.8943264764432648
Type 1 error:  0.19708029197080293 | False Positive: predicted customer accepted the campaign, while it was not!
Type 2 error:  0.13636363636363635 | False Negative - predicted response customer do not accepted campaign, while it was!


In [ ]:
# display model scores for premium cluster
display_scores(lreg_stan, Xtest_stan, ytest_stan)

Accuracy:  0.7633587786259542
AUC:  0.8865546218487395
Type 1 error:  0.23949579831932774 | False Positive: predicted customer accepted the campaign, while it was not!
Type 2 error:  0.20833333333333334 | False Negative - predicted response customer do not accepted campaign, while it was!


### Feature importance

In [ ]:
def get_feature_importance_plot(lreg, Xtrain, ytrain):
  # get features used in modeling
  features_list = Xtrain.columns

  # calculate feat importance
  feature_imp_list = lreg.coef_
  feature_imp_list = list(feature_imp_list.flatten())

  # create plot df
  feat_importance_df = pd.DataFrame({
                                   'feature':features_list,
                                   'feature_importance':feature_imp_list
                                   })
  # round values
  feat_importance_df['feature_importance'] = feat_importance_df['feature_importance'].round(2)
  # create absolute column to sort
  feat_importance_df['feature_importance_abs'] = feat_importance_df['feature_importance'].abs()
  # sort values by absolute value of feature importance
  feat_importance_df = feat_importance_df.sort_values(by = 'feature_importance_abs', ascending = False)

  # plot feature importance bar chart
  fig = px.bar(
              feat_importance_df,
              x = 'feature',
              y = 'feature_importance',
              text_auto=True,
              color_discrete_sequence = [primary_color],
              title = 'Feature importance for model prediction',
              height = 700,
              width = 1600
              )
  fig = fig.update_layout(
                          font_size = 14,
                          title_font_size = 20,
                          plot_bgcolor = third_color
                          )
  return fig

In [ ]:
# feature importance for premium cluster
get_feature_importance_plot(lreg_prem, Xtrain_prem, ytrain_prem)

In [ ]:
# feature importance for standard cluster
get_feature_importance_plot(lreg_stan, Xtrain_stan, ytrain_stan)

## Estimate profit

Now it's time for us to estimate the financial impact of segmentation and understand how effective it was.

Considering we have

**y = rpc * (sr/100) - cpc**

where:

y = profit per client (variable)

rpc = revenuae per client that accepted the offer (in our case, constant =  11)

sr = sucess rate (variable)

cpc = cost per client that recieved the offer (in our case, constant = 3)

In [ ]:
# create x and y values to plot
x_list = range(1,101)
y_list = []
for sr in range(1,101):
  y = 11 * (sr/100) - 3
  y_list.append(y)

In [ ]:
# plot sucess rate x profit per/customer chart
fig = px.line(
              x = x_list,
              y = y_list)

# change axis names and background color to match presentation pattern
fig = fig.update_layout(
                        yaxis_title= 'Profit Per Customer',
                        xaxis_title = 'Campaign Sucess Rate',
                        plot_bgcolor = third_color,
                        )

# change line color and width to match presentation pattern
fig = fig.update_traces(line_color = primary_color, line_width=3)

# add an horizontal line that traces break even point
fig = fig.add_hline(y=0, opacity=0.5, line_dash='dash')
fig